# <font color='red'>2. Spark and transformers</font>
Transfomers are the core of Spark "thinking" process. For better understanding it is best to start at begining: <b>How does Spark actually compute things in the code</b>?

load the data

In [0]:
input_data = spark.read.parquet("/FileStore/cleanData")
input_data.createOrReplaceTempView("input_data")

## <font color='red'>2.1 Transformers</font>
So far we only did simple transformation query in SQL. But Spark can do more, much more actually.

**Documentation:** https://spark.apache.org/docs/2.3.0/ml-features.html

This is where <b><font color='red'>Transformes</font></b> come in. Transfomer is an object with two functions you can call:
- **Fit** (optional) - here are the data, look at them, find your parameters but **dont give me anything**
- **Transform** (always present) - here are the data, apply your transformation and **give me the result**

Let's further explain this using the first real transformer - **Imputer**

### 2.2.1 Imputer
Imputer is transformer for filling out missing numerical values using mean or median.

**Example**: Let's say we have house Id without PriceNtile defined. Our decision (in reality not optimal) is that we want to fill these rows with median.

In [0]:
from pyspark.ml.feature import Imputer
imputer = Imputer(inputCols=['PriceNtile']
                  , outputCols=['PriceNtile_filled']
                  , strategy ="median")

If we now call **.fit(dataset)** on this imputer, Spark finds what is the median of YearBuilt, but does not change data or return anything.

In [0]:
imputer_fitted = imputer.fit(input_data)

print(imputer_fitted)

ImputerModel: uid=Imputer_11aabdbf50bd, strategy=median, missingValue=NaN, numInputCols=1, numOutputCols=1

If we want to actually get the dataframe with a new column with the filled values (Spark in most cases creates new column rather than changing existing) we call **.transform(dataset)**. This returns the dataframe with a new column.

In [0]:
data_imputed = imputer_fitted.transform(input_data)

data_imputed.select("Id", "PriceNtile", "PriceNtile_filled").show()

+----+----------+-----------------+
 Id|PriceNtile|PriceNtile_filled|
+----+----------+-----------------+
 496| 1.0| 1.0|
 917| 1.0| 1.0|
 969| 1.0| 1.0|
 534| 1.0| 1.0|
 31| 1.0| 1.0|
 711| 1.0| 1.0|
1338| 1.0| 1.0|
 706| 1.0| 1.0|
1326| 1.0| 1.0|
 813| 1.0| 1.0|
1381| 1.0| 1.0|
 411| 1.0| 1.0|
 637| 1.0| 1.0|
1101| 1.0| 1.0|
 376| 1.0| 1.0|
 463| 1.0| 1.0|
1433| 1.0| 1.0|
 875| 1.0| 1.0|
 399| 1.0| 1.0|
 621| 1.0| 1.0|
+----+----------+-----------------+
only showing top 20 rows

### 2.2.2 SQLTransformer
SQLTransformer lets you perform any SQL code on dataset send to it. It has only **.transform(dataset)** method as there are no parameters it needs 'find' for itself.

**Example:** Let's generalize outlier removal from first section to any dataset send to it.

In [0]:
from pyspark.ml.feature import SQLTransformer

sqlTransformer = SQLTransformer()\
  .setStatement("""
      SELECT *,
        case 
            when LotArea < 1450.0 then 1450.0 
            when LotArea > 17690.0 then 17690.0 
            else LotArea 
          end LotArea_bound
        , case 
            when YearBuilt < 1885.0 then 1885.0 
            when YearBuilt > 2069.0 then 2069.0 
            else YearBuilt 
          end YearBuilt_bound
        , Id
        , "ahoj " as const
      FROM __THIS__
    """)

** '__THIS__' ** is table you always select from, it will be the dataset you send to **.transform(dataset)** method.

Let's see the results:

In [0]:
data_transformed = sqlTransformer.transform(input_data)

data_transformed.select("Id", "LotArea","LotArea_bound", "YearBuilt", "YearBuilt_bound", "const").show(10, False)

+----+-------+-------------+---------+---------------+-----+
Id |LotArea|LotArea_bound|YearBuilt|YearBuilt_bound|const|
+----+-------+-------------+---------+---------------+-----+
496 |7879 |7879.0 |1920 |1920.0 |ahoj |
917 |9000 |9000.0 |1949 |1949.0 |ahoj |
969 |5925 |5925.0 |1910 |1910.0 |ahoj |
534 |5000 |5000.0 |1946 |1946.0 |ahoj |
31 |8500 |8500.0 |1920 |1920.0 |ahoj |
711 |4130 |4130.0 |1935 |1935.0 |ahoj |
1338|4118 |4118.0 |1941 |1941.0 |ahoj |
706 |5600 |5600.0 |1930 |1930.0 |ahoj |
1326|3636 |3636.0 |1922 |1922.0 |ahoj |
813 |8712 |8712.0 |1952 |1952.0 |ahoj |
+----+-------+-------------+---------+---------------+-----+
only showing top 10 rows

### 2.2.3 QuantileDiscretizer
Splits a continious numerical variable into n buckets, based on quantiles.

**Example:** Let's split SalePrice column into 6 buckets.

In [0]:
from pyspark.ml.feature import QuantileDiscretizer

quantile_discretizer = QuantileDiscretizer(inputCol='SalePrice'
                  , outputCol='SalePrice_bucket'
                  , numBuckets = 6)

Lets fit, transform and seee the output:

In [0]:
quantile_discretizer\
.fit(input_data)\
.transform(input_data)\
.select("Id", "SalePrice", "SalePrice_bucket")\
.show()

+----+---------+----------------+
 Id|SalePrice|SalePrice_bucket|
+----+---------+----------------+
 496| 34900| 0.0|
 917| 35311| 0.0|
 969| 37900| 0.0|
 534| 39300| 0.0|
 31| 40000| 0.0|
 711| 52000| 0.0|
1338| 52500| 0.0|
 706| 55000| 0.0|
1326| 55000| 0.0|
 813| 55993| 0.0|
1381| 58500| 0.0|
 411| 60000| 0.0|
 637| 60000| 0.0|
1101| 60000| 0.0|
 376| 61000| 0.0|
 463| 62383| 0.0|
1433| 64500| 0.0|
 875| 66500| 0.0|
 399| 67000| 0.0|
 621| 67000| 0.0|
+----+---------+----------------+
only showing top 20 rows

### 2.2.4 StringIndexer

A label indexer that maps a string column of labels to an ML column of label indices. If the input column is numeric, we cast it to string and index the string values. The indices are in [0], numLabels), ordered by label frequencies. So the most frequent label gets index 0.
The simplest way to index is via the StringIndexer, which maps strings to different numerical IDs. Spark’s StringIndexer also creates metadata attached to the DataFrame that specify what inputs correspond to what outputs. This allows us later to get inputs back from their respective index values:

handleInvalid:
 - skip (which will filter out rows with bad values)
 - error (which will throw an error)
 - keep (creates new index)
 
**Example:** Let's index categorical column MSSubClass.

In [0]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol = "MSSubClass"
                            ,outputCol='MSSubClass_index'
                            ,handleInvalid="keep")

And see the result:

In [0]:
data_indexed = indexer.fit(input_data).transform(input_data)

data_indexed \
    .select("Id", "MSSubClass", "MSSubClass_index") \
    .show()

+----+----------+----------------+
 Id|MSSubClass|MSSubClass_index|
+----+----------+----------------+
 496| 30| 4.0|
 917| 20| 0.0|
 969| 50| 2.0|
 534| 20| 0.0|
 31| 70| 6.0|
 711| 30| 4.0|
1338| 30| 4.0|
 706| 190| 9.0|
1326| 30| 4.0|
 813| 20| 0.0|
1381| 30| 4.0|
 411| 20| 0.0|
 637| 30| 4.0|
1101| 30| 4.0|
 376| 30| 4.0|
 463| 20| 0.0|
1433| 30| 4.0|
 875| 50| 2.0|
 399| 30| 4.0|
 621| 30| 4.0|
+----+----------+----------------+
only showing top 20 rows

### 2.2.5 OneHotEncoderEstimator

  - Indexing categorical variables is only half of the story. One-hot encoding is an extremely common data transformation performed after indexing categorical variables. This is because indexing does not always represent our categorical variables in the correct way for downstream models to process. For instance, when we index our “color” column, you will notice that some colors have a higher value (or index number) than others (in our case, blue is 1 and green is 2).
  - This is incorrect because it gives the mathematical appearance that the input to the machine learning algorithm seems to specify that green > blue, which makes no sense in the case of the current categories. To avoid this, we use OneHotEncoder, which will convert each distinct value to a Boolean flag (1 or 0) as a component in a vector. When we encode the color value, then we can see these are no longer ordered, making them easier for downstream models (e.g., a linear model) to process
  - Input column must be of type NumericType
  
**Example:** One hot encode our indexed variable MSSubClass_indexed.

In [0]:
from pyspark.ml.feature import OneHotEncoder

encoder = OneHotEncoder(inputCols = ["MSSubClass_index"],
                                 outputCols=["MSSubClass_index_vec"])

And see the result:

In [0]:
encoder \
    .fit(data_indexed) \
    .transform(data_indexed)\
    .select("Id", "MSSubClass", "MSSubClass_index", "MSSubClass_index_vec") \
    .show()

+----+----------+----------------+--------------------+
 Id|MSSubClass|MSSubClass_index|MSSubClass_index_vec|
+----+----------+----------------+--------------------+
 496| 30| 4.0| (15,[4],[1.0])|
 917| 20| 0.0| (15,[0],[1.0])|
 969| 50| 2.0| (15,[2],[1.0])|
 534| 20| 0.0| (15,[0],[1.0])|
 31| 70| 6.0| (15,[6],[1.0])|
 711| 30| 4.0| (15,[4],[1.0])|
1338| 30| 4.0| (15,[4],[1.0])|
 706| 190| 9.0| (15,[9],[1.0])|
1326| 30| 4.0| (15,[4],[1.0])|
 813| 20| 0.0| (15,[0],[1.0])|
1381| 30| 4.0| (15,[4],[1.0])|
 411| 20| 0.0| (15,[0],[1.0])|
 637| 30| 4.0| (15,[4],[1.0])|
1101| 30| 4.0| (15,[4],[1.0])|
 376| 30| 4.0| (15,[4],[1.0])|
 463| 20| 0.0| (15,[0],[1.0])|
1433| 30| 4.0| (15,[4],[1.0])|
 875| 50| 2.0| (15,[2],[1.0])|
 399| 30| 4.0| (15,[4],[1.0])|
 621| 30| 4.0| (15,[4],[1.0])|
+----+----------+----------------+--------------------+
only showing top 20 rows

### 2.2.6 VectorAssembler
 - One of the most important transforms in Spark. Most models do not take as an input a list of columns and what to predict, but they need one vector with information from all columns.
 - VectorAssembler takes as an input the list of columns a returns a new column with a vector.
  
**Example:** Join three numerical columns LotArea, YearBuilt, YearRemodAdd into vector.

In [0]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=["LotArea", "YearBuilt", "YearRemodAdd"], 
                            outputCol="Num_vector")

See the output:

In [0]:
assembler \
    .transform(input_data)\
    .select("Id", "LotArea", "YearBuilt", "YearRemodAdd", "Num_vector") \
    .show(20, False)

+----+-------+---------+------------+-----------------------+
Id |LotArea|YearBuilt|YearRemodAdd|Num_vector |
+----+-------+---------+------------+-----------------------+
496 |7879 |1920 |1950 |[7879.0,1920.0,1950.0] |
917 |9000 |1949 |1950 |[9000.0,1949.0,1950.0] |
969 |5925 |1910 |1950 |[5925.0,1910.0,1950.0] |
534 |5000 |1946 |1950 |[5000.0,1946.0,1950.0] |
31 |8500 |1920 |1950 |[8500.0,1920.0,1950.0] |
711 |4130 |1935 |2003 |[4130.0,1935.0,2003.0] |
1338|4118 |1941 |1950 |[4118.0,1941.0,1950.0] |
706 |5600 |1930 |1950 |[5600.0,1930.0,1950.0] |
1326|3636 |1922 |1950 |[3636.0,1922.0,1950.0] |
813 |8712 |1952 |1952 |[8712.0,1952.0,1952.0] |
1381|8212 |1914 |1950 |[8212.0,1914.0,1950.0] |
411 |9571 |1958 |1958 |[9571.0,1958.0,1958.0] |
637 |6120 |1936 |1950 |[6120.0,1936.0,1950.0] |
1101|8400 |1920 |1950 |[8400.0,1920.0,1950.0] |
376 |10020 |1922 |1950 |[10020.0,1922.0,1950.0]|
463 |8281 |1965 |1965 |[8281.0,1965.0,1965.0] |
1433|10800 |1927 |2007 |[10800.0,1927.0,2007.0]|
875 |5720 |1941 |1950 |[5720.0,1941.0,1950.0] |
399 |8967 |1920 |1950 |[8967.0,1920.0,1950.0] |
621 |8248 |1914 |1950 |[8248.0,1914.0,1950.0] |
+----+-------+---------+------------+-----------------------+
only showing top 20 rows